# Structured query language (SQL)

<img src="../figures/allenembedding.png" >

- Generating queries that will be run based on natural language questions
- Creating chatbots that can answer questions based on database data
- Building custom dashboards based on insights a user wants to analyze

## LangChain provides tools to interact with SQL Databases:

1. `Build SQL` queries based on natural language user questions
2. `Query a SQL database` using chains for query creation and execution
3. `Interact with a SQL database` using agents for robust and flexible querying

In [1]:
# !pip install langchain langchain-experimental openai
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI
# Load environment variables from .env file
load_dotenv()
api_key = os.getenv("OPEN_AI_KEY")
api_organization = os.getenv("OPEN_AI_ORG")

llm = OpenAI(
    api_key= api_key,
    openai_organization= api_organization,
    temperature=0, 
    verbose=True
)

## Case 1: Text-to-SQL query

In [10]:
from langchain.prompts.prompt import PromptTemplate

PROMPT_SUFFIX = """Only use the following tables:
{table_info}

Question: {input}"""

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

"""

PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect", "top_k"],
    template=_DEFAULT_TEMPLATE + PROMPT_SUFFIX,
)

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///./docs/sql/Chinook.db")
db.get_table_names()

/opt/homebrew/lib/python3.11/site-packages/langchain/utilities/sql_database.py:278: UserWarning: This method is deprecated - please use `get_usable_table_names`.
  warnings.warn(


['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///./docs/sql/Chinook.db")

chain = create_sql_query_chain(
    llm = llm, 
    db = db,
    # You can add prompt
)

response = chain.invoke(
    {"question": "How many employees are there"}
)

print(response)

SELECT COUNT(*) FROM Employee


In [6]:
from langchain.prompts import PromptTemplate

TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Some examples of SQL queries that correspond to questions are:

{few_shot_examples}

Question: {input}"""

CUSTOM_PROMPT = PromptTemplate(
    input_variables=["input", "few_shot_examples", "table_info", "dialect"],
    template=TEMPLATE,
)

In [7]:
# !pip3 install langchainhub

In [24]:
# from langchain import hub

# CUSTOM_PROMPT = hub.pull("rlm/text-to-sql")
# CUSTOM_PROMPT

## Case 2: Text-to-SQL query and execution

The performance of the `SQLDatabaseChain` can be enhanced in several ways:

- `Adding sample rows`
- `Specifying custom table information`
- `Using Query Checker` self-correct invalid SQL using parameter use_query_checker=True
- `Customizing the LLM Prompt` include specific instructions or relevant information, using parameter prompt=CUSTOM_PROMPT
- `Get intermediate steps` access the SQL statement as well as the final result using parameter return_intermediate_steps=True
- `Limit the number of rows` a query will return using parameter top_k=5

In [3]:
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase.from_uri("sqlite:///../docs/sql/Chinook.db")

llm = OpenAI(
    api_key= api_key,
    openai_organization= api_organization,
    temperature=0, 
    verbose=True
)

db_chain = SQLDatabaseChain.from_llm(
    llm = llm, 
    db = db, 
    verbose=True,
)

In [4]:
db_chain.run("How many employees are there?")



> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT COUNT(*) FROM "Employee";
SQLResult: [(8,)]
Answer:There are 8 employees.
> Finished chain.


'There are 8 employees.'

## Case 3: SQL agents

In [16]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

db = SQLDatabase.from_uri("sqlite:///./docs/sql/Chinook.db")

llm = OpenAI(
    api_key= api_key,
    openai_organization= api_organization,
    temperature=0, 
    verbose=True
)

toolkit = SQLDatabaseToolkit(
    db=db, 
    llm=llm
),

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [17]:
# Agent task example #1 - Running queries
agent_executor.run(
    "List the total sales per country. Which country's customers spent the most?"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: I should query the schema of the Invoice and Customer tables.
Action: sql_db_schema
Action Input: Invoice, Customer
Observation: 
CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa

'The customers from the USA spent the most, with a total of $523.06.'

In [18]:
# Agent task example #2 - Describing a Table
agent_executor.run("Describe the playlisttrack table")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: The PlaylistTrack table is the most relevant to the question.
Action: sql_db_schema
Action Input: PlaylistTrack
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought: I now know the final answer
Final Answer: The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers and form a primary key. It also has two foreign keys, one to the Track table and one to the Playlist table.

> Finished chain.


'The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers and form a primary key. It also has two foreign keys, one to the Track table and one to the Playlist table.'

## Extending the SQL Toolkit

Including dynamic few-shot examples
In order to include dynamic few-shot examples, we need a custom Retriever Tool that handles the vector database in order to retrieve the examples that are semantically similar to the user’s question.

Let's start by creating a dictionary with some examples:

In [17]:
db = SQLDatabase.from_uri(
	"sqlite:///./docs/sql/Chinook.db",
	include_tables=['Track'], # including only one table for illustration
	sample_rows_in_table_info=3
)

print(db.table_info)


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

/*
3 rows from Track table:
TrackId	Name	AlbumId	MediaTypeId	GenreId	Composer	Milliseconds	Bytes	UnitPrice
1	For Those About To Rock (We Salute You)	1	1	1	Angus Young, Malcolm Young, Brian Johnson	343719	11170334	0.99
2	Balls to the Wall	2	2	1	None	342562	5510424	0.99
3	Fast As a Shark	3	2	1	F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman	230619	3990994	0.99
*/


In [3]:
few_shots = {
    "List all artists.": "SELECT * FROM artists;",
    "Find all albums for the artist 'AC/DC'.": "SELECT * FROM albums WHERE ArtistId = (SELECT ArtistId FROM artists WHERE Name = 'AC/DC');",
    "List all tracks in the 'Rock' genre.": "SELECT * FROM tracks WHERE GenreId = (SELECT GenreId FROM genres WHERE Name = 'Rock');",
    "Find the total duration of all tracks.": "SELECT SUM(Milliseconds) FROM tracks;",
    "List all customers from Canada.": "SELECT * FROM customers WHERE Country = 'Canada';",
    "How many tracks are there in the album with ID 5?": "SELECT COUNT(*) FROM tracks WHERE AlbumId = 5;",
    "Find the total number of invoices.": "SELECT COUNT(*) FROM invoices;",
    "List all tracks that are longer than 5 minutes.": "SELECT * FROM tracks WHERE Milliseconds > 300000;",
    "Who are the top 5 customers by total purchase?": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM invoices GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    "Which albums are from the year 2000?": "SELECT * FROM albums WHERE strftime('%Y', ReleaseDate) = '2000';",
    "How many employees are there": 'SELECT COUNT(*) FROM "employee"',
}

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()

few_shot_docs = [
    Document(page_content=question, metadata={"sql_query": few_shots[question]})
    for question in few_shots.keys()
]
vector_db = FAISS.from_documents(few_shot_docs, embeddings)
retriever = vector_db.as_retriever()

In [7]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool_description = """
This tool will help you understand similar examples to adapt them to the user question.
Input to this tool should be the user question.
"""

retriever_tool = create_retriever_tool(
    retriever, name="sql_get_similar_examples", description=tool_description
)
custom_tool_list = [retriever_tool]
custom_tool_list

[Tool(name='sql_get_similar_examples', description='\nThis tool will help you understand similar examples to adapt them to the user question.\nInput to this tool should be the user question.\n', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x11873d650>)>, coroutine=<bound method BaseRetriever.aget_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x11873d650>)>)]

In [15]:
from langchain.agents import AgentType, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///./docs/sql/Chinook.db")

llm = ChatOpenAI(
    api_key= api_key,
    openai_organization= api_organization,
    temperature=0, 
    verbose=True
)

toolkit = SQLDatabaseToolkit(
    db=db, 
    llm=llm
)

custom_suffix = """
I should first get the similar examples I know.
If the examples are enough to construct the query, I can build it.
Otherwise, I can then look at the tables in the database to see what I can query.
Then I should query the schema of the most relevant tables
"""

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    extra_tools=custom_tool_list,
    suffix=custom_suffix,
    
)

In [16]:
agent.run("How many employees do we have?")



> Entering new AgentExecutor chain...

Invoking: `sql_get_similar_examples` with `{'query': 'How many employees are there?'}`


[Document(page_content='How many employees are there', metadata={'sql_query': 'SELECT COUNT(*) FROM "employee"'}), Document(page_content='Find the total number of invoices.', metadata={'sql_query': 'SELECT COUNT(*) FROM invoices;'}), Document(page_content='Who are the top 5 customers by total purchase?', metadata={'sql_query': 'SELECT CustomerId, SUM(Total) AS TotalPurchase FROM invoices GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;'}), Document(page_content='How many tracks are there in the album with ID 5?', metadata={'sql_query': 'SELECT COUNT(*) FROM tracks WHERE AlbumId = 5;'})]
Invoking: `sql_db_query` with `SELECT COUNT(*) FROM employee`
responded: Based on similar examples, I can construct the following query to find the number of employees:

```sql
SELECT COUNT(*) FROM employee;
```

Let me execute the query to get the answer.

[(8,)]We h

'We have 8 employees.'

## Multiple Tool 

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.chains import LLMMathChain
from pydantic import BaseModel, Field

class CalculatorInput(BaseModel):
    question: str = Field()

db = SQLDatabase.from_uri("sqlite:///./docs/sql/Chinook.db")

db_chain = SQLDatabaseChain.from_llm(
    llm = llm, 
    db = db, 
    verbose=True,
)

calculator = LLMMathChain.from_llm(
    llm=llm,
    verbose=True
)

tools = [
    Tool(
        name="Database",
        func=db_chain.run,
        description="useful for when you need to answer questions about database",
    ),
    Tool(
        name="Calculator",
        func=calculator.run,
        description="useful when you need to do math operations or arithmetic.",
        args_schema=CalculatorInput,
    ),
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent.run("How many employees do we have?")



> Entering new AgentExecutor chain...
 I need to access a database to answer this question.
Action: Database
Action Input: Employee count

> Entering new SQLDatabaseChain chain...
Employee count
SQLQuery:SELECT COUNT(*) FROM "Employee"
SQLResult: [(8,)]
Answer:8 employees
> Finished chain.

Observation: 8 employees
Thought: I now know the final answer.
Final Answer: We have 8 employees.

> Finished chain.


'We have 8 employees.'

## Reference 
- [SQL Langchain](https://python.langchain.com/docs/use_cases/qa_structured/sql)